In [1]:
import tensorflow.keras.backend as K
import numpy as np
import pandas as pd
import json
import multiprocessing
import matplotlib.pyplot as plt

from glob import glob
from copy import deepcopy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
n_cores = multiprocessing.cpu_count()
n_cores

16

In [3]:
from fl_tissue_model_tools import data_prep, dev_config, models, defs
import fl_tissue_model_tools.preprocessing as prep

In [4]:
dirs = dev_config.get_dev_directories("../dev_paths.txt")

# Set up model training parameters

In [5]:
with open("../model_training/invasion_depth_training_values.json", 'r') as fp:
    training_values = json.load(fp)
training_values["rs_seed"] = None if (training_values["rs_seed"] == "None") else training_values["rs_seed"]

In [6]:
training_values

{'batch_size': 32,
 'frozen_epochs': 50,
 'fine_tune_epochs': 50,
 'val_split': 0.2,
 'early_stopping_patience': 25,
 'early_stopping_min_delta': 0.0001,
 'rs_seed': None}

In [7]:
with open("../model_training/invasion_depth_best_hp.json", 'r') as fp:
    best_hp = json.load(fp)

In [8]:
best_hp

{'adam_beta_1': 0.9066466810625207,
 'adam_beta_2': 0.9947698608592066,
 'fine_tune_lr': 9.141354728081903e-05,
 'frozen_lr': 0.00022767552973325001,
 'last_resnet_layer': 'conv5_block1_out'}

In [9]:
### Data paths ###
root_data_path = f"{dirs.data_dir}/invasion_data/"
model_training_path = f"{dirs.analysis_dir}/resnet50_invasion_model"
best_ensemble_training_path = f"{model_training_path}/best_ensemble"


### Model building & training parameters ###
resnet_inp_shape = (128, 128, 3)
class_labels = {"no_invasion": 0, "invasion": 1}

# Binary classification -> only need 1 output unit
n_outputs = 1
seed = training_values["rs_seed"]
val_split = training_values["val_split"]
batch_size = training_values["batch_size"]
frozen_epochs = training_values["frozen_epochs"]
fine_tune_epochs = training_values["fine_tune_epochs"]
# frozen_epochs = 5
# fine_tune_epochs = 5
adam_beta_1 = best_hp["adam_beta_1"]
adam_beta_2 = best_hp["adam_beta_2"]
frozen_lr = best_hp["frozen_lr"]
fine_tune_lr = best_hp["fine_tune_lr"]
last_resnet_layer = best_hp["last_resnet_layer"]


### Early stopping ###
es_criterion = "val_loss"
es_mode = "min"
# Update these depending on seriousness of experiment
es_patience = training_values["early_stopping_patience"]
es_min_delta = training_values["early_stopping_min_delta"]


### Model saving ###
mcp_criterion = "val_loss"
mcp_mode = "min"
mcp_best_only = True
# Need to set to True otherwise base model "layer" won't save/load properly
mcp_weights_only = True

In [10]:
data_prep.make_dir(best_ensemble_training_path)

In [11]:
rs = np.random.RandomState(seed)

# Train ensemble of models using best hyperparameters

In [12]:
n_models = 5

In [13]:
for model_idx in range(n_models):
    ### Prepare data (each model should be trained on a randomly assigned train/validation set) ###
    # Training & validation data (drawn from same image set & randomly assigned)
    tv_class_paths = {v: glob(f"{root_data_path}/train/{k}/*.tif") for k, v in class_labels.items()}
    mcp_best_frozen_weights_file = f"{best_ensemble_training_path}/best_frozen_weights_{model_idx}.h5"
    mcp_best_finetune_weights_file = f"{best_ensemble_training_path}/best_finetune_weights_{model_idx}.h5"

    for k, v in tv_class_paths.items():
        rs.shuffle(v)

    train_data_paths, val_data_paths = data_prep.get_train_val_split(tv_class_paths, val_split=val_split)


    ### Build datasets ###
    train_datagen = data_prep.InvasionDataGenerator(
        train_data_paths,
        class_labels,
        batch_size,
        resnet_inp_shape[:2],
        rs,
        class_weights=True,
        shuffle=True,
        augmentation_function=prep.augment_imgs
    )
    val_datagen = data_prep.InvasionDataGenerator(
        val_data_paths,
        class_labels,
        batch_size,
        resnet_inp_shape[:2],
        rs,
        class_weights=train_datagen.class_weights,
        shuffle=True,
        augmentation_function=train_datagen.augmentation_function
    )


    ### Build model ###
    K.clear_session()
    tl_model = models.build_ResNet50_TL(
        n_outputs,
        resnet_inp_shape,
        base_last_layer=last_resnet_layer,
        base_model_trainable=False
    )


    ### Frozen training ###
    tl_model.compile(
        optimizer=Adam(learning_rate=frozen_lr, beta_1=adam_beta_1, beta_2=adam_beta_2),
        loss=BinaryCrossentropy(),
        weighted_metrics=[BinaryAccuracy()]
    )

    es_callback = EarlyStopping(monitor=es_criterion, mode=es_mode, min_delta=es_min_delta, patience=es_patience)
    mcp_callback = ModelCheckpoint(mcp_best_frozen_weights_file, monitor=mcp_criterion, mode=mcp_mode, save_best_only=mcp_best_only, save_weights_only=mcp_weights_only)

    h1 = tl_model.fit(
        train_datagen,
        validation_data=val_datagen,
        epochs=frozen_epochs,
        callbacks=[es_callback, mcp_callback],
        workers=n_cores
    )


    ### Finetune training ###
    # Load best frozen weights before fine tuning
    tl_model.load_weights(mcp_best_frozen_weights_file)
    # Make base model trainable
    models.toggle_TL_freeze(tl_model)
    tl_model.compile(
        optimizer=Adam(learning_rate=fine_tune_lr, beta_1=adam_beta_1, beta_2=adam_beta_2),
        loss=BinaryCrossentropy(),
        weighted_metrics=[BinaryAccuracy()]
    )

    es_callback = EarlyStopping(monitor=es_criterion, mode=es_mode, min_delta=es_min_delta, patience=es_patience)
    mcp_callback = ModelCheckpoint(mcp_best_finetune_weights_file, monitor=mcp_criterion, mode=mcp_mode, save_best_only=mcp_best_only, save_weights_only=mcp_weights_only)

    h2 = tl_model.fit(
        train_datagen,
        validation_data=val_datagen,
        epochs=fine_tune_epochs,
        callbacks=[es_callback, mcp_callback],
        workers=n_cores
    )


    ### Save results for comparison later ###
    h1_df = pd.DataFrame(h1.history)
    h1_df["training_stage"] = ["frozen"] * len(h1.epoch)

    h2_df = pd.DataFrame(h2.history)
    h2_df["training_stage"] = ["finetune"] * len(h2.epoch)

    h_df = pd.concat([h1_df, h2_df], axis=0, ignore_index=True)

    h_df.to_csv(f"{best_ensemble_training_path}/best_model_history_{model_idx}.csv", index=False)

Epoch 1/5
16/16 [==============================] - 13s 310ms/step - loss: 0.7148 - binary_accuracy: 0.5553 - val_loss: 0.6388 - val_binary_accuracy: 0.6125
Epoch 2/5
16/16 [==============================] - 5s 224ms/step - loss: 0.6100 - binary_accuracy: 0.6551 - val_loss: 0.5786 - val_binary_accuracy: 0.7489
Epoch 3/5
16/16 [==============================] - 5s 217ms/step - loss: 0.5361 - binary_accuracy: 0.7393 - val_loss: 0.5006 - val_binary_accuracy: 0.7998
Epoch 4/5
16/16 [==============================] - 6s 237ms/step - loss: 0.5050 - binary_accuracy: 0.7284 - val_loss: 0.4913 - val_binary_accuracy: 0.7281
Epoch 5/5
16/16 [==============================] - 5s 218ms/step - loss: 0.4797 - binary_accuracy: 0.7914 - val_loss: 0.4995 - val_binary_accuracy: 0.7750
Epoch 1/5
16/16 [==============================] - 14s 357ms/step - loss: 0.6812 - binary_accuracy: 0.7228 - val_loss: 0.4500 - val_binary_accuracy: 0.7783
Epoch 2/5
16/16 [==============================] - 6s 262ms/step - l

KeyboardInterrupt: 